In [1]:
import numpy as np
from dmrghandler.src.dmrghandler.qchem_dmrg_calc import single_qchem_dmrg_calc
from dmrghandler.src.dmrghandler.dmrg_looping import dmrg_central_loop
import dmrghandler.src.dmrghandler.dmrg_calc_prepare as dmrg_calc_prepare
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from block2 import SZ as block2_SZ
from get_dmrg_config import get_dmrg_process_param, get_dmrg_config
from pathlib import Path
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock2 import tools
from pyblock2._pyscf.ao2mo import integrals as itg
import pyscf
from OrbitalRotation.eigenstate_check import check_eigenstates
from basic_definitions import chemist_to_physicist, get_ferm_op, physicist_to_chemist
from Hamiltonians.molecular_hamiltonians import get_n2, get_h2o, get_hf, get_he, get_co, get_h2
import openfermion as of
from Hamiltonians.dmrg_hamiltonians import get_dmrg_hamiltonian, dmrg_non_exact

In [3]:
from pyscf import gto, scf

mol = gto.M(
    atom=f"O 0 0 0; H 0 0 1; H 0 1 0",
    basis='sto-3g')
mf = scf.RHF(mol).run(conv_tol=1E-14)
mycc = mf.CISD().run()
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
                                                                     ncore=0,
                                                                     g2e_symm=1)

print(f"SCF energy: {mf.kernel() - ecore}")
print(f"CISD energy: {mycc.e_tot - ecore}")

converged SCF energy = -74.9611711378677
E(RCISD) = -75.01826559023739  E_corr = -0.05709445236967557
converged SCF energy = -74.9611711378677
SCF energy: -83.80219130687863
CISD energy: -83.85928575924831


In [4]:
n_elec

10

In [5]:
num_orbitals = ncas
num_electrons = n_elec
num_unpaired = spin
multiplicity = 2 * spin + 1

dmrg_params = get_dmrg_config(
    num_orbitals, num_electrons, num_unpaired,
    multiplicity)

### Given h1e, g2e, the following returns the physicist 
absorbed_tbt = pyscf.fci.direct_nosym.absorb_h1e(np.array(h1e.copy()), np.array(g2e.copy()), norb=num_orbitals, nelec=num_electrons, fac=0.5)
h_in_tbt = pyscf.ao2mo.restore(1, absorbed_tbt.copy(), num_orbitals).astype(np.array(h1e.copy()).dtype, copy=False)
one_body_dummy = np.zeros((ncas, ncas))
obt, tbt = chemist_to_physicist(one_body_dummy, h_in_tbt)

label = "_original"
filename = "fcidump.H2O" + f"{label}"
pyscf.tools.fcidump.from_integrals(
    "fcidump_output"/Path(filename),
    obt,
    tbt,
    nmo=obt.shape[0],
    nelec=n_elec,
    nuc=ecore,
    ms=spin,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

In [73]:
import shutil
import os

# Define the directory path
dir_path = "./mps_directory"
dir_path2 = "./mps_directory_2"

# Check if the directory exists
if os.path.exists(dir_path):
    # Delete the directory and all its contents
    shutil.rmtree(dir_path)
    print(f"Deleted '{dir_path}' and all its contents.")
else:
    print(f"Directory '{dir_path}' does not exist.")
    
if os.path.exists(dir_path2):
    # Delete the directory and all its contents
    shutil.rmtree(dir_path2)
    print(f"Deleted '{dir_path2}' and all its contents.")
else:
    print(f"Directory '{dir_path2}' does not exist.")

Deleted './mps_directory' and all its contents.
Deleted './mps_directory_2' and all its contents.


In [74]:
tbt[0, 0, 9, 9]

-1.886021524575402

In [75]:
loop_results = dmrg_central_loop(obt, tbt, dmrg_params, 1000, 1e6, 1e-10   , "./mps_directory", verbosity=1, move_mps_to_final_storage_path=None)


integral cutoff error =  0.0
mpo terms =      17994

Build MPO | Nsites =    10 | Nterms =      17994 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /    10 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.001 T = 0.006
 Site =     1 /    10 .. Mmpo =    82 DW = 0.00e+00 NNZ =      479 SPT = 0.8053 Tmvc = 0.002 T = 0.007
 Site =     2 /    10 .. Mmpo =   126 DW = 0.00e+00 NNZ =      939 SPT = 0.9091 Tmvc = 0.002 T = 0.007
 Site =     3 /    10 .. Mmpo =   186 DW = 0.00e+00 NNZ =     1579 SPT = 0.9326 Tmvc = 0.002 T = 0.008
 Site =     4 /    10 .. Mmpo =   262 DW = 0.00e+00 NNZ =     2203 SPT = 0.9548 Tmvc = 0.002 T = 0.010
 Site =     5 /    10 .. Mmpo =   186 DW = 0.00e+00 NNZ =     8147 SPT = 0.8328 Tmvc = 0.003 T = 0.037
 Site =     6 /    10 .. Mmpo =   126 DW = 0.00e+00 NNZ =     1575 SPT = 0.9328 Tmvc = 0.001 T = 0.008
 Site =     7 /    10 .. Mmpo =    82 DW = 0.00e+00 NNZ =      943 SPT = 0.9087 Tmvc = 0.000 T = 0.004
 Site =     8 /    10 .. Mmpo =

In [76]:
active_indices = [3, 4, 5, 6]
N = len(active_indices)
active_space_tbt = np.zeros((N, N, N, N))
for p in range(N):
    for q in range(N):
        for r in range(N):
            for s in range(N):
                active_space_tbt[p, q, r, s] = h_in_tbt[active_indices[p], active_indices[q], active_indices[r], active_indices[s]]


tensor_matrix = active_space_tbt.reshape(N**2, N**2)

eigenvalues, unitary_matrix_large = np.linalg.eigh(tensor_matrix)

# Step 3: Extract an N x N unitary matrix from the large unitary
# Reshape the large unitary (N^2, N^2) matrix back to (N, N, N, N) and average over pairs of indices
# This gives an effective N x N unitary matrix
unitary_matrix_N = np.zeros((N, N))

for i in range(N):
    for j in range(N):
        # Take the block from the larger unitary matrix that corresponds to the i, j pair
        block = unitary_matrix_large[i*N:(i+1)*N, j*N:(j+1)*N]
        # Average the block or take the dominant block
        unitary_matrix_N[i, j] = np.mean(block)

U_NO = unitary_matrix_N
print(unitary_matrix_N @ unitary_matrix_N.conj().T)
# Step 4: Apply this N x N unitary matrix to diagonalize the original tensor
# Transform the tensor by applying the unitary to each index
p = np.einsum_path('ak,bl,cm,dn,klmn->abcd', U_NO, U_NO, U_NO, U_NO, active_space_tbt)[0]
orbital_rotated = np.einsum('ak,bl,cm,dn,klmn->abcd', U_NO, U_NO, U_NO, U_NO, active_space_tbt, optimize = p)




[[ 0.00185614 -0.0031903   0.003391    0.00244833]
 [-0.0031903   0.03453741 -0.00768812 -0.00805423]
 [ 0.003391   -0.00768812  0.01204539  0.00508452]
 [ 0.00244833 -0.00805423  0.00508452  0.0044507 ]]


In [77]:
from scipy.linalg import svd

# Assuming h_in_tbt and active_indices are defined, populating the tensor
for p in range(N):
    for q in range(N):
        for r in range(N):
            for s in range(N):
                active_space_tbt[p, q, r, s] = h_in_tbt.copy()[active_indices[p], active_indices[q], active_indices[r], active_indices[s]]

# HOSVD decomposition for each mode (p, q, r, s)
# We will perform SVD on the unfoldings of the tensor for each mode

# Step 1: Unfold the tensor along each mode and perform SVD
def unfold(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1))

# Perform SVD for each mode
U_p, _, _ = svd(unfold(active_space_tbt, 0), full_matrices=False)
U_q, _, _ = svd(unfold(active_space_tbt, 1), full_matrices=False)
U_r, _, _ = svd(unfold(active_space_tbt, 2), full_matrices=False)
U_s, _, _ = svd(unfold(active_space_tbt, 3), full_matrices=False)

U_combined = (U_p + U_q + U_r + U_s) / 4

# Ensure U_combined is unitary using QR decomposition or SVD again
U_combined, _ = np.linalg.qr(U_combined)  # Ensures U_combined is unitary

# Step 3: Apply the combined unitary to the original tensor
# Transform the tensor by applying the unitary to all modes
diagonalized_tensor = np.einsum('pa,qb,rc,sd,pqrs->abcd', U_combined, U_combined,
                                U_combined, U_combined, active_space_tbt)

# Step 4: Verify unitarity of the combined matrix
print("Is U_combined unitary?", np.allclose(np.dot(U_combined, U_combined.conj().T), np.eye(N)))

# Print the diagonalized tensor
print("Diagonalized Tensor:")
print(diagonalized_tensor)

print("Combined Unitary Matrix U_combined (N x N):")
print(U_combined)

Is U_combined unitary? True
Diagonalized Tensor:
[[[[-3.05227508e-01  3.18753617e-11  3.05716574e-16 -3.03035899e-16]
   [ 3.18753668e-11 -2.89573188e-01  1.10856217e-17 -8.19914986e-18]
   [ 3.05716574e-16  1.10856217e-17 -2.89237713e-01  2.77019932e-22]
   [-3.03035899e-16 -8.19914986e-18 -6.93861688e-18 -2.89237713e-01]]

  [[ 3.18753690e-11  4.27275794e-02  5.81532614e-17 -4.51280586e-17]
   [ 4.27275794e-02 -2.48270717e-12 -6.40704547e-15  6.28123750e-15]
   [ 5.81532614e-17 -6.40704547e-15 -1.10286105e-11 -2.11857294e-12]
   [-4.51280586e-17  6.28123750e-15 -2.11857294e-12 -1.47799443e-11]]

  [[ 3.05716574e-16  5.81532614e-17  3.58651594e-02  4.33682795e-18]
   [ 5.81532614e-17 -1.62579287e-15  2.59984665e-12 -3.41337047e-13]
   [ 3.58651594e-02  2.59984665e-12  3.55201872e-17  3.56417390e-17]
   [ 4.33682834e-18 -3.41337047e-13  3.56417390e-17  1.05176396e-16]]

  [[-3.03035899e-16 -4.51280586e-17  3.46948109e-18  3.58651594e-02]
   [-4.51280586e-17  1.59638820e-15 -3.41337422e

In [78]:
diagonalized_tensor[0, 0, 3, 3]

-0.2892377131560469

In [79]:
transform_unitary = np.identity(ncas)
for p in range(N):
    for q in range(N):
        transform_unitary[active_indices[p], active_indices[q]] = U_combined[p, q]
        
U_NO = transform_unitary

In [80]:
p = np.einsum_path('ak,bl,cm,dn,klmn->abcd', U_NO, U_NO, U_NO, U_NO, h_in_tbt)[0]
orbital_rotated = np.einsum('ak,bl,cm,dn,klmn->abcd', U_NO, U_NO, U_NO, U_NO, h_in_tbt, optimize = p)

In [81]:
orbital_rotated[3, 3, 3, 4]

4.5096214143172753e-11

In [82]:
U = np.eye(ncas)
U[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], :] = U[[3, 4, 0, 1, 2, 7, 8, 9, 5, 6], :]
U_NO = U

In [83]:
p = np.einsum_path('ak,bl,cm,dn,klmn->abcd', U_NO, U_NO, U_NO, U_NO, orbital_rotated)[0]
orbital_rotated_perm = np.einsum('ak,bl,cm,dn,klmn->abcd', U_NO, U_NO, U_NO, U_NO, orbital_rotated, optimize = p)

In [84]:
num_orbitals = ncas
num_electrons = n_elec
num_unpaired = spin
multiplicity = 2 * spin + 1

dmrg_params = get_dmrg_config(
    num_orbitals, num_electrons, num_unpaired,
    multiplicity)
one_body_dummy = np.zeros((ncas, ncas))
obt, tbt = chemist_to_physicist(one_body_dummy, orbital_rotated_perm)

In [85]:
# label = "_active_space3"
# filename = "fcidump.N2_22" + f"{label}"
# pyscf.tools.fcidump.from_integrals(
#     "fcidump_output"/Path(filename),
#     obt,
#     tbt,
#     nmo=obt.shape[0],
#     nelec=n_elec,
#     nuc=ecore,
#     ms=spin,
#     orbsym=None,
#     tol=1E-8,
#     float_format=' %.16g',
# )


In [86]:
n = obt.shape[0]
for p in range(n):
    for q in range(n):
        if abs(obt[p, q]) < 1e-8:
            obt[p, q] = 0
        for r in range(n):
            for s in range(n):
                if abs(tbt[p, q, r, s]) < 1e-8:
                    tbt[p, q, r, s] = 0

In [87]:
loop_results = dmrg_central_loop(obt, tbt, dmrg_params, 1000, 1e6, 1e-10   , "./mps_directory_2", verbosity=1, move_mps_to_final_storage_path=None)


integral cutoff error =  0.0
mpo terms =       4520

Build MPO | Nsites =    10 | Nterms =       4520 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /    10 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.001 T = 0.004
 Site =     1 /    10 .. Mmpo =    66 DW = 0.00e+00 NNZ =      175 SPT = 0.9116 Tmvc = 0.000 T = 0.002
 Site =     2 /    10 .. Mmpo =   110 DW = 0.00e+00 NNZ =      403 SPT = 0.9445 Tmvc = 0.000 T = 0.002
 Site =     3 /    10 .. Mmpo =   170 DW = 0.00e+00 NNZ =      519 SPT = 0.9722 Tmvc = 0.000 T = 0.004
 Site =     4 /    10 .. Mmpo =   150 DW = 0.00e+00 NNZ =     2337 SPT = 0.9084 Tmvc = 0.000 T = 0.003
 Site =     5 /    10 .. Mmpo =   162 DW = 0.00e+00 NNZ =      399 SPT = 0.9836 Tmvc = 0.000 T = 0.001
 Site =     6 /    10 .. Mmpo =   106 DW = 0.00e+00 NNZ =      569 SPT = 0.9669 Tmvc = 0.000 T = 0.001
 Site =     7 /    10 .. Mmpo =    50 DW = 0.00e+00 NNZ =      207 SPT = 0.9609 Tmvc = 0.000 T = 0.001
 Site =     8 /    10 .. Mmpo =

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       4520

Build MPO | Nsites =    10 | Nterms =       4520 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /    10 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /    10 .. Mmpo =    66 DW = 0.00e+00 NNZ =      175 SPT = 0.9116 Tmvc = 0.000 T = 0.001
 Site =     2 /    10 .. Mmpo =   110 DW = 0.00e+00 NNZ =      403 SPT = 0.9445 Tmvc = 0.000 T = 0.002
 Site =     3 /    10 .. Mmpo =   170 DW = 0.00e+00 NNZ =      519 SPT = 0.9722 Tmvc = 0.000 T = 0.002
 Site =     4 /    10 .. Mmpo =   150 DW = 0.00e+00 NNZ =     2337 SPT = 0.9084 Tmvc = 0.000 T = 0.003
 Site =     5 /    10 .. Mmpo =   162 DW = 0.00e+00 NNZ =      399 SPT = 0.9836 Tmvc = 0.000 T = 0.001
 Site =     6 /    10 .. Mmpo =   106 DW = 0.00e+00 NNZ =      569 SPT = 0.9669 Tmvc = 0.000 T = 0.001
 Site =     7 /    10 .. Mmpo =    50 DW = 0.00e+00 NNZ =      207 SPT = 0.9609 Tmvc = 0.000 T = 0.001
 Site =     8 /    10 .. Mmpo =

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       4520

Build MPO | Nsites =    10 | Nterms =       4520 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /    10 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /    10 .. Mmpo =    66 DW = 0.00e+00 NNZ =      175 SPT = 0.9116 Tmvc = 0.000 T = 0.002
 Site =     2 /    10 .. Mmpo =   110 DW = 0.00e+00 NNZ =      403 SPT = 0.9445 Tmvc = 0.000 T = 0.002
 Site =     3 /    10 .. Mmpo =   170 DW = 0.00e+00 NNZ =      519 SPT = 0.9722 Tmvc = 0.000 T = 0.002
 Site =     4 /    10 .. Mmpo =   150 DW = 0.00e+00 NNZ =     2337 SPT = 0.9084 Tmvc = 0.000 T = 0.003
 Site =     5 /    10 .. Mmpo =   162 DW = 0.00e+00 NNZ =      399 SPT = 0.9836 Tmvc = 0.000 T = 0.001
 Site =     6 /    10 .. Mmpo =   106 DW = 0.00e+00 NNZ =      569 SPT = 0.9669 Tmvc = 0.000 T = 0.001
 Site =     7 /    10 .. Mmpo =    50 DW = 0.00e+00 NNZ =      207 SPT = 0.9609 Tmvc = 0.000 T = 0.001
 Site =     8 /    10 .. Mmpo =

...

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       4520

Build MPO | Nsites =    10 | Nterms =       4520 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /    10 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.001 T = 0.004
 Site =     1 /    10 .. Mmpo =    66 DW = 0.00e+00 NNZ =      175 SPT = 0.9116 Tmvc = 0.000 T = 0.002
 Site =     2 /    10 .. Mmpo =   110 DW = 0.00e+00 NNZ =      403 SPT = 0.9445 Tmvc = 0.000 T = 0.003
 Site =     3 /    10 .. Mmpo =   170 DW = 0.00e+00 NNZ =      519 SPT = 0.9722 Tmvc = 0.001 T = 0.004
 Site =     4 /    10 .. Mmpo =   150 DW = 0.00e+00 NNZ =     2337 SPT = 0.9084 Tmvc = 0.000 T = 0.004
 Site =     5 /    10 .. Mmpo =   162 DW = 0.00e+00 NNZ =      399 SPT = 0.9836 Tmvc = 0.000 T = 0.006
 Site =     6 /    10 .. Mmpo =   106 DW = 0.00e+00 NNZ =      569 SPT = 0.9669 Tmvc = 0.000 T = 0.004
 Site =     7 /    10 .. Mmpo =    50 DW = 0.00e+00 NNZ =      207 SPT = 0.9609 Tmvc = 0.000 T = 0.003
 Site =     8 /    10 .. Mmpo =

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       4520

Build MPO | Nsites =    10 | Nterms =       4520 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /    10 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /    10 .. Mmpo =    66 DW = 0.00e+00 NNZ =      175 SPT = 0.9116 Tmvc = 0.000 T = 0.001
 Site =     2 /    10 .. Mmpo =   110 DW = 0.00e+00 NNZ =      403 SPT = 0.9445 Tmvc = 0.000 T = 0.002
 Site =     3 /    10 .. Mmpo =   170 DW = 0.00e+00 NNZ =      519 SPT = 0.9722 Tmvc = 0.000 T = 0.002
 Site =     4 /    10 .. Mmpo =   150 DW = 0.00e+00 NNZ =     2337 SPT = 0.9084 Tmvc = 0.000 T = 0.003
 Site =     5 /    10 .. Mmpo =   162 DW = 0.00e+00 NNZ =      399 SPT = 0.9836 Tmvc = 0.000 T = 0.001
 Site =     6 /    10 .. Mmpo =   106 DW = 0.00e+00 NNZ =      569 SPT = 0.9669 Tmvc = 0.000 T = 0.001
 Site =     7 /    10 .. Mmpo =    50 DW = 0.00e+00 NNZ =      207 SPT = 0.9609 Tmvc = 0.000 T = 0.001
 Site =     8 /    10 .. Mmpo =

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


In [41]:
print(obt[1, 8])

IndexError: index 8 is out of bounds for axis 1 with size 7